In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 9s 162ms/step - loss: 0.6659 - accuracy: 0.5783 - precision: 0.6091 - recall: 0.2804 - auc: 0.6455
Epoch 2/30
58/58 [==============================] - 9s 160ms/step - loss: 0.5620 - accuracy: 0.7119 - precision: 0.6813 - recall: 0.7243 - auc: 0.7803
Epoch 3/30
58/58 [==============================] - 10s 171ms/step - loss: 0.4833 - accuracy: 0.7875 - precision: 0.7511 - recall: 0.8178 - auc: 0.8408
Epoch 4/30
58/58 [==============================] - 10s 176ms/step - loss: 0.5032 - accuracy: 0.7525 - precision: 0.6898 - recall: 0.8575 - auc: 0.8299
Epoch 5/30
58/58 [==============================] - 10s 176ms/step - loss: 0.4500 - accuracy: 0.7930 - precision: 0.7424 - recall: 0.8551 - auc: 0.8712
Epoch 6/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4207 - accuracy: 0.8138 - precision: 0.7756 - recall: 0.8481 - auc: 0.8882
Epoch 7/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4025 - accura

58/58 [==============================] - 10s 179ms/step - loss: 0.2763 - accuracy: 0.8981 - precision_1: 0.8714 - recall_1: 0.9182 - auc_1: 0.9458
Epoch 24/30
58/58 [==============================] - 10s 177ms/step - loss: 0.3029 - accuracy: 0.8762 - precision_1: 0.8261 - recall_1: 0.9322 - auc_1: 0.9416
Epoch 25/30
58/58 [==============================] - 10s 178ms/step - loss: 0.4031 - accuracy: 0.8215 - precision_1: 0.8132 - recall_1: 0.8037 - auc_1: 0.9012
Epoch 26/30
58/58 [==============================] - 10s 178ms/step - loss: 0.2721 - accuracy: 0.8949 - precision_1: 0.8756 - recall_1: 0.9042 - auc_1: 0.9514
Epoch 27/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2415 - accuracy: 0.9102 - precision_1: 0.9005 - recall_1: 0.9089 - auc_1: 0.9598
Epoch 28/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2473 - accuracy: 0.9135 - precision_1: 0.8852 - recall_1: 0.9369 - auc_1: 0.9562
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 10s 179ms/step - loss: 0.3835 - accuracy: 0.8620 - precision_3: 0.8386 - recall_3: 0.8738 - auc_3: 0.9084
Epoch 15/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2686 - accuracy: 0.8949 - precision_3: 0.8790 - recall_3: 0.8995 - auc_3: 0.9509
Epoch 16/30
58/58 [==============================] - 10s 177ms/step - loss: 0.2540 - accuracy: 0.9146 - precision_3: 0.9248 - recall_3: 0.8902 - auc_3: 0.9594
Epoch 17/30
58/58 [==============================] - 10s 177ms/step - loss: 0.3889 - accuracy: 0.8357 - precision_3: 0.8233 - recall_3: 0.8271 - auc_3: 0.9049
Epoch 18/30
58/58 [==============================] - 10s 179ms/step - loss: 0.2796 - accuracy: 0.8938 - precision_3: 0.8894 - recall_3: 0.8832 - auc_3: 0.9525
Epoch 19/30
58/58 [==============================] - 10s 178ms/step - loss: 0.2005 - accuracy: 0.9310 - precision_3: 0.9274 - recall_3: 0.9252 - auc_3: 0.9732
Epoch 20/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 312ms/step - loss: 0.6611 - accuracy: 0.5805 - precision_5: 0.5856 - recall_5: 0.3598 - auc_5: 0.6273
Epoch 2/30
58/58 [==============================] - 18s 315ms/step - loss: 0.5268 - accuracy: 0.7404 - precision_5: 0.7054 - recall_5: 0.7664 - auc_5: 0.8096
Epoch 3/30
58/58 [==============================] - 18s 309ms/step - loss: 0.4656 - accuracy: 0.7963 - precision_5: 0.7725 - recall_5: 0.8014 - auc_5: 0.8632
Epoch 4/30
58/58 [==============================] - 18s 313ms/step - loss: 0.4377 - accuracy: 0.8039 - precision_5: 0.7701 - recall_5: 0.8294 - auc_5: 0.8759
Epoch 5/30
58/58 [==============================] - 18s 311ms/step - loss: 0.3676 - accuracy: 0.8598 - precision_5: 0.8348 - recall_5: 0.8738 - auc_5: 0.9140
Epoch 6/30
58/58 [==============================] - 18s 310ms/step - loss: 0.3512 - accuracy: 0.8543 - precision_5: 0.8406 - recall_5: 0.8505 - auc_5: 0.9227
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 313ms/step - loss: 0.1822 - accuracy: 0.9398 - precision_6: 0.9191 - recall_6: 0.9556 - auc_6: 0.9766
Epoch 23/30
58/58 [==============================] - 18s 309ms/step - loss: 0.2478 - accuracy: 0.9211 - precision_6: 0.8921 - recall_6: 0.9463 - auc_6: 0.9573
Epoch 24/30
58/58 [==============================] - 18s 313ms/step - loss: 0.1282 - accuracy: 0.9639 - precision_6: 0.9478 - recall_6: 0.9766 - auc_6: 0.9886
Epoch 25/30
58/58 [==============================] - 18s 311ms/step - loss: 0.1178 - accuracy: 0.9693 - precision_6: 0.9566 - recall_6: 0.9790 - auc_6: 0.9875
Epoch 26/30
58/58 [==============================] - 18s 314ms/step - loss: 0.1030 - accuracy: 0.9715 - precision_6: 0.9653 - recall_6: 0.9743 - auc_6: 0.9900
Epoch 27/30
58/58 [==============================] - 18s 312ms/step - loss: 0.1342 - accuracy: 0.9463 - precision_6: 0.9317 - recall_6: 0.9556 - auc_6: 0.9878
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 18s 313ms/step - loss: 0.5521 - accuracy: 0.7295 - precision_8: 0.7819 - recall_8: 0.5864 - auc_8: 0.7903
Epoch 14/30
58/58 [==============================] - 18s 318ms/step - loss: 0.3724 - accuracy: 0.8488 - precision_8: 0.8571 - recall_8: 0.8131 - auc_8: 0.9143
Epoch 15/30
58/58 [==============================] - 18s 314ms/step - loss: 0.3158 - accuracy: 0.8719 - precision_8: 0.8977 - recall_8: 0.8201 - auc_8: 0.9397
Epoch 16/30
58/58 [==============================] - 18s 313ms/step - loss: 0.2547 - accuracy: 0.9124 - precision_8: 0.8973 - recall_8: 0.9182 - auc_8: 0.9573
Epoch 17/30
58/58 [==============================] - 18s 314ms/step - loss: 0.2397 - accuracy: 0.9102 - precision_8: 0.8968 - recall_8: 0.9136 - auc_8: 0.9608
Epoch 18/30
58/58 [==============================] - 18s 310ms/step - loss: 0.2168 - accuracy: 0.9233 - precision_8: 0.8960 - recall_8: 0.9463 - auc_8: 0.9665
Epoch 19/30
58/58 [==============================] - 18s 3